# Supervised NLP
## Using Gutenberg corpora - Alice in Wonderland / Persuasion
Will Becerra

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

c:\users\will\projects\environments\project1_env\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\will\projects\environments\project1_env\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
c:\users\will\projects\environments\project1_env\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\will\projects\environments\project1_env\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


We are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. 

Our feature-generation approach will be Bag of Words. For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
import nltk
# Launch the installer to download "gutenberg" and "stop words" corpora.
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [4]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [5]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


spaCy has already tokenized and labelled our data, so we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  We'll work with lemmas (root words) rather than the raw text terms, and we'll only use the 500 most common words for each text.

In [12]:
# Utility function to create a list of the 500 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(500)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
      
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]

        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1

        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [13]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,duchess,bottle,small,oop,need,interesting,heart,oblige,share,say,...,then,aware,ought,bath,sir,run,with,would,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's test the bag of words using a Random Forest.

In [14]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9768025078369906

Test set score: 0.8782894736842105


Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [15]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 774) (3190,)
Training set score: 0.9423197492163009

Test set score: 0.9163533834586466


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [16]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8874608150470219

Test set score: 0.8745300751879699


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [17]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [18]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [19]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [20]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [21]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.691648023862789


col_0,Austen,Carroll
row_0,,
Austen,1561,108
Carroll,719,294


NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# BoW with KNN

In [22]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
train = knn.fit(X_train, y_train)

print('Training set score:', knn.score(X_train, y_train))
print('\nTest set score:', knn.score(X_test, y_test))

Training set score: 0.8711598746081505

Test set score: 0.8054511278195489


# BoW with Support Vector Machine

In [25]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

svc = svm.SVC(kernel ='linear')

# Specify hyperparameter range
C = [0.01, 0.1, 1, 10]

# Set hyperparameters 
hyperparameters = dict(C=C)

# Create grid search using 5-fold cross validation
svc_clf = GridSearchCV(svc, hyperparameters, cv=5)

# Run grid searchb
svc_results = svc_clf.fit(X_train, y_train)

# Get results
print('C:', svc_results.best_estimator_.get_params()['C'])
print('Best score:', round(svc_results.best_score_, 3))

print('Training set score:', svc_results.score(X_train, y_train))
print('\nTest set score:', svc_results.score(X_test, y_test))

C: 1
Best score: 0.898
Training set score: 0.9501567398119122

Test set score: 0.9102443609022557


## SVM turns out to score best, above the threshold of 90%, without major overfitting.

In [28]:
# Below, I back up some DataFrames in case the notebook crashes.

In [23]:
word_counts.to_csv('word_nlp.csv', sep=',')

In [24]:
emma_bow.to_csv('emma_nlp.csv', sep=',')

In [61]:
word_counts = pd.read_csv('word_nlp.csv')

In [62]:
word_counts.head()

,Unnamed: 0,duchess,bottle,small,oop,need,interesting,heart,oblige,share,...,then,aware,ought,bath,sir,run,with,would,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Alice was beginning to get very tired of sitti...,Carroll
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,So she was considering in her own mind (as wel...,Carroll
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,There was nothing so VERY remarkable in that; ...,Carroll
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Oh dear!,Carroll
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,I shall be late!',Carroll


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [31]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [45]:
# Clean the Sense & Sensibility data.
sense = gutenberg.raw('austen-sense.txt')
sense = re.sub(r'VOLUME \w+', '', sense)
sense = re.sub(r'CHAPTER \w+', '', sense)
sense = text_cleaner(sense)
print(sense[:100])

The family of Dashwood had long been settled in Sussex. Their estate was large, and their residence 


In [46]:
# Parse the cleaned novel.
sense_doc = nlp(sense)

In [47]:
# Group into sentences.
sense_sents = [[sent, "Austen"] for sent in sense_doc.sents]

In [48]:
len(alice_sents)

1669

In [49]:
len(sense_sents)

5219

In [50]:
# Sense and Sensibility is quite long, let's cut it down to the same length as Alice.
sense_sents = sense_sents[0:len(alice_sents)]

In [52]:
# Build a new Bag of Words data frame for Sense and Sensibility word counts.
# We'll use the same common words from Alice and Persuasion.
sense_sentences = pd.DataFrame(sense_sents)
sense_bow = bow_features(sense_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [53]:
# Now we'll model it using logistic regression.

# Combine the Sense and Sensibility sentence data with the Alice data from the test set.
X_Sense_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    sense_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Sense_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * sense_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Sense_test, y_Sense_test))
lr_Sense_predicted = lr.predict(X_Sense_test)
pd.crosstab(y_Sense_test, lr_Sense_predicted)


Test set score: 0.674123788217748


col_0,Austen,Carroll
row_0,,
Austen,1514,155
Carroll,719,294


## This model is able to differentiate multiple works of Austen from Carroll.

## Let's try working with The Parent's Assistant by Edgeworth, comparing it to Carroll.

In [36]:
# Clean The Parent's Assistant data.
parents = gutenberg.raw('edgeworth-parents.txt')
parents = re.sub(r'VOLUME \w+', '', parents)
parents = re.sub(r'CHAPTER \w+', '', parents)
parents = text_cleaner(parents)
print(parents[:100])

THE ORPHANS. Near the ruins of the castle of Rossmore, in Ireland, is a small cabin, in which there 


In [ ]:
# Parse the cleaned novel.
parents_doc = nlp(parents)

In [54]:
# Group into sentences.
parents_sents = [[sent, "Edgeworth"] for sent in parents_doc.sents]

In [48]:
len(alice_sents)

1669

In [55]:
len(parents_sents)

10208

In [56]:
# The Parent's Assistant is quite long, let's cut it down to the same length as Alice.
parents_sents = parents_sents[0:len(alice_sents)]

In [63]:
# Combine the sentences from the two novels into one data frame.
sentences_2 = pd.DataFrame(alice_sents + parents_sents)
sentences_2.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [57]:
parentswords = bag_of_words(parents_doc)

In [58]:
# Combine bags to create a set of unique words.
common_words_2 = set(alicewords + parentswords)

In [64]:
# Create our data frame with features. This can take a while to run.
word_counts_2 = bow_features(sentences_2, common_words_2)
word_counts_2.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000


,duchess,bottle,small,oop,need,interesting,oblige,heart,say,hand,...,well,footman,then,ought,sir,run,with,would,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Let's try Random Forest first

In [65]:
rfc = ensemble.RandomForestClassifier()
Y_2 = word_counts_2['text_source']
X_2 = np.array(word_counts_2.drop(['text_sentence','text_source'], 1))

X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, 
                                                    Y_2,
                                                    test_size=0.4,
                                                    random_state=0)
train_2 = rfc.fit(X_2_train, y_2_train)

print('Training set score:', rfc.score(X_2_train, y_2_train))
print('\nTest set score:', rfc.score(X_2_test, y_2_test))

Training set score: 0.975024975024975

Test set score: 0.8196107784431138


## Since SVM performed best, let's try it again.

In [66]:
svc = svm.SVC(kernel ='linear')

# Specify hyperparameter range
C = [0.01, 0.1, 1, 10]

# Set hyperparameters 
hyperparameters = dict(C=C)

# Create grid search using 5-fold cross validation
svc_clf_2 = GridSearchCV(svc, hyperparameters, cv=5)

# Run grid searchb
svc_results_2 = svc_clf_2.fit(X_2_train, y_2_train)

# Get results
print('C:', svc_results_2.best_estimator_.get_params()['C'])
print('Best score:', round(svc_results_2.best_score_, 3))

print('Training set score:', svc_results_2.score(X_2_train, y_2_train))
print('\nTest set score:', svc_results_2.score(X_2_test, y_2_test))

C: 0.1
Best score: 0.849
Training set score: 0.9005994005994006

Test set score: 0.8398203592814372


## Using SVM on Alice and Parents shows the model is good at identifying the difference between works.